In [1]:
!pip install music21

In [2]:
import glob
import pickle
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
import keras.utils
from keras.callbacks import ModelCheckpoint

In [18]:
notes_train = []
durations_train = [] 

notes_val = []
durations_val = [] 

train_song = {}
val_song = {}

dir_path = "musicnet_midis"
for folder in os.listdir(dir_path):
    ts, vs = train_test_split(os.listdir(f"{dir_path}/{folder}"), test_size=0.2, random_state=27)
    train_song[str(folder)] = ts
    val_song[str(folder)] = vs

In [22]:
folder = os.listdir(dir_path)

for fold in folder:
    for song in train_song[str(fold)]:
        try:
            # print(file)
            midi = converter.parse(f"musicnet_midis/{fold}/{song}")
            notes_to_parse = None
            parts = instrument.partitionByInstrument(midi)

            if parts:  # File has instrument parts
                notes_to_parse = parts.parts[0].recurse()
            else:  # File has notes in a flat structure
                notes_to_parse = midi.flat.notes

            prev_offset = 0.0  # Keep track of the previous note's offset

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes_train.append(str(element.pitch))
                    duration = element.duration.quarterLength  # Get note duration
                    durations_train.append(duration)  # Append duration to the durations list
                elif isinstance(element, chord.Chord):
                    notes_train.append('.'.join(str(n) for n in element.normalOrder))
    #                 print('.'.join(str(n) for n in element.normalOrder))
                    duration = element.duration.quarterLength  # Get chord duration
                    durations_train.append(duration)  # Append duration to the durations list

                    # Assuming all notes in the chord have the same duration
                    prev_offset = element.offset + element.duration.quarterLength
        except Exception as e:
            print(f"Error processing file: {song} - {e}")
            pass

/opt/conda/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=17, channel=None, data=b'Copyright \xa92007 by'>; getting generic Instrument
  warnings.warn(


Error processing file: Ravel - index out of range
Error processing file: Ravel - badly formed midi string: missing leading MTrk
Error processing file: Ravel - badly formed midi string: missing leading MTrk
Error processing file: Ravel - index out of range
Error processing file: Ravel - badly formed midi string: missing leading MTrk
Error processing file: Ravel - badly formed midi string: missing leading MTrk
Error processing file: Ravel - badly formed midi string: missing leading MTrk


/opt/conda/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=13, channel=None, data=b'Copyright \xa92002 by'>; getting generic Instrument
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Trio f\xfcr Violine, Cello und Klavier Op.99, Rondo'>; getting generic Instrument
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Trio f\xfcr Klarinette, Cello und Klavier Op.38, Adagio'>; getting generic Instrument
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <mu

In [23]:
folder = os.listdir(dir_path)

for fold in folder:
    for song in val_song[str(fold)]:
        try:
            # print(file)
            midi = converter.parse(f"musicnet_midis/{fold}/{song}")
            notes_to_parse = None
            parts = instrument.partitionByInstrument(midi)

            if parts:  # File has instrument parts
                notes_to_parse = parts.parts[0].recurse()
            else:  # File has notes in a flat structure
                notes_to_parse = midi.flat.notes

            prev_offset = 0.0  # Keep track of the previous note's offset

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes_val.append(str(element.pitch))
                    duration = element.duration.quarterLength  # Get note duration
                    durations_val.append(duration)  # Append duration to the durations list
                elif isinstance(element, chord.Chord):
                    notes_val.append('.'.join(str(n) for n in element.normalOrder))
    #                 print('.'.join(str(n) for n in element.normalOrder))
                    duration = element.duration.quarterLength  # Get chord duration
                    durations_val.append(duration)  # Append duration to the durations list

                    # Assuming all notes in the chord have the same duration
                    prev_offset = element.offset + element.duration.quarterLength
        except Exception as e:
            print(f"Error processing file: {song} - {e}")
            pass

/opt/conda/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Andante con variazioni dalla Sonata n\xb09 Op.47'>; getting generic Instrument
  warnings.warn(


In [26]:
with open('notes_train.txt', 'w') as notes_file:
    for note in notes_train:
        notes_file.write(f"{note}\n")

with open('notes_val.txt', 'w') as notes_file:
    for note in notes_val:
        notes_file.write(f"{note}\n")

with open('durations_train.txt', 'w') as durations_file:
    for duration in durations_train:
        durations_file.write(f"{duration}\n")
        
with open('durations_val.txt', 'w') as durations_file:
    for duration in durations_val:
        durations_file.write(f"{duration}\n")
    

In [4]:
sequence_length = 25

# Combine pitch and duration into tuples
combined_notes_durations = list(zip(notes_train, durations_train))

# get all unique combined pitch and duration pairs
unique_pairs = sorted(set(item for item in combined_notes_durations))

# create a dictionary to map unique pairs to integers
pair_to_int = {pair: number for number, pair in enumerate(unique_pairs)}
pair_to_int[("oov","oov")] = len(unique_pairs)

network_input = []
network_output = []

# create input sequences and corresponding outputs
for i in range(0, len(combined_notes_durations) - sequence_length, 1):
    sequence_in = combined_notes_durations[i:i + sequence_length]
    sequence_out = combined_notes_durations[i + sequence_length]

    # map pitches and durations to their integer representations
    network_input.append([pair_to_int[pair] for pair in sequence_in])
    network_output.append(pair_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input
network_input = network_input / float(len(unique_pairs)+1)

# one-hot encode the output (since it represents both pitch and duration)
network_output = keras.utils.to_categorical(network_output, num_classes=len(unique_pairs)+1)

In [5]:
len(pair_to_int)

3017

In [59]:
sequence_length = 25

# Combine pitch and duration into tuples
combined_notes_durations_val = list(zip(notes_val, durations_val))

# get all unique combined pitch and duration pairs
unique_pairs_val = sorted(set(item for item in combined_notes_durations_val))

# create a dictionary to map unique pairs to integers
# pair_to_int = {pair: number for number, pair in enumerate(unique_pairs)}

network_input_val = []
network_output_val = []

# create input sequences and corresponding outputs
for i in range(0, len(combined_notes_durations_val) - sequence_length, 1):
    sequence_in_val = combined_notes_durations_val[i:i + sequence_length]
    sequence_out_val = combined_notes_durations_val[i + sequence_length]

    # map pitches and durations to their integer representations
    sequence_int_val = []
    for pair in sequence_in_val:
        # Check if the pair exists in pair_to_int, if not, use ("oov", "oov")
        if pair in pair_to_int:
            sequence_int_val.append(pair_to_int[pair])
        else:
            sequence_int_val.append(pair_to_int[("oov", "oov")])

    network_input_val.append(sequence_int_val)
#     network_input_val.append([pair_to_int[pair] for pair in sequence_in_val])
    try:
        network_output_val.append(pair_to_int[sequence_out_val])
    except:
        network_output_val.append(pair_to_int[("oov", "oov")])
#     network_output_val.append(pair_to_int[sequence_out_val])

n_patterns_val = len(network_input_val)

# reshape the input into a format compatible with LSTM layers
network_input_val = np.reshape(network_input_val, (n_patterns_val, sequence_length, 1))

# normalize input
network_input_val = network_input_val / float(len(unique_pairs)+1)

# one-hot encode the output (since it represents both pitch and duration)
network_output_val = keras.utils.to_categorical(network_output_val, num_classes=len(unique_pairs)+1)

In [63]:
n_vocab= len(unique_pairs)+1

model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [64]:
filepath = "firstmodel-{epoch:02d}-{loss:.4f}-bigger.h5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=3, batch_size=64, callbacks=callbacks_list)

Epoch 1/3
4489/4489 [==============================] - 123s 25ms/step - loss: 6.2898 - accuracy: 0.0136
Epoch 2/3
   1/4489 [..............................] - ETA: 1:43 - loss: 6.2316 - accuracy: 0.0156

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4489/4489 [==============================] - 116s 26ms/step - loss: 6.2191 - accuracy: 0.0185
Epoch 3/3
4489/4489 [==============================] - 116s 26ms/step - loss: 6.1557 - accuracy: 0.0241


In [ ]:
from music21 import note, stream, duration, midi

# Create a reverse dictionary to map integers back to combined note-duration pairs
int_to_pair = {number: pair for pair, number in pair_to_int.items()}

start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

prediction_output = []

# generate notes
for note_index in range(100):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    
    result = int_to_pair[index]  # Retrieve the note-duration pair from the integer index
    prediction_output.append(result)
    
    # Update pattern for the next iteration
    to_append = index / float(len(unique_pairs))
    pattern = np.append(pattern, [[to_append]], axis=0)
    pattern = pattern[1:len(pattern)]

In [8]:
prediction_output

[('2.4', 0.25),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.4.7', 1.0),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2', 0.25),
 ('0.2',

In [9]:
output_stream = stream.Stream()

# Create note and duration objects and append them to output_notes
for pattern in prediction_output:
    note_pitch, note_duration = pattern

    # Check if the note_pitch is a float (representing a note duration) and handle accordingly
    if isinstance(note_pitch, float):
        # Assuming the previous note was a chord, end it and start a new note with a duration
        output_stream[-1].duration.quarterLength = note_pitch
    else:
        if '.' in note_pitch:  # If it's a chord
            notes_in_chord = note_pitch.split('.')
            chord_notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.duration.quarterLength = note_duration
                chord_notes.append(new_note)
            new_chord = chord.Chord(chord_notes)
            output_stream.append(new_chord)
        else:  # If it's a single note
            new_note = note.Note(note_pitch)
            new_note.duration.quarterLength = note_duration
            output_stream.append(new_note)

# Write the MIDI file
midi_stream = midi.translate.music21ObjectToMidiFile(output_stream)
midi_stream.open('generated_music1.mid', 'wb')
midi_stream.write()
midi_stream.close()